<a href="https://colab.research.google.com/github/linqus/nlp-huggingface/blob/main/notebooks/unit5/nlp_5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install requests datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.7 MB/s eta 0:00:00


In [2]:
import requests

url = "https://api.github.com/repos/huggingface/datasets/issues?page=1&per_page=1"
response = requests.get(url)

In [3]:
response.status_code

200

In [5]:
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/6484',
  'repository_url': 'https://api.github.com/repos/huggingface/datasets',
  'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/6484/labels{/name}',
  'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/6484/comments',
  'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/6484/events',
  'html_url': 'https://github.com/huggingface/datasets/issues/6484',
  'id': 2033333294,
  'node_id': 'I_kwDODunzps55MjQu',
  'number': 6484,
  'title': '[Feature Request] Dataset versioning',
  'user': {'login': 'kenfus',
   'id': 47979198,
   'node_id': 'MDQ6VXNlcjQ3OTc5MTk4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/47979198?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/kenfus',
   'html_url': 'https://github.com/kenfus',
   'followers_url': 'https://api.github.com/users/kenfus/followers',
   'following_url': 'https://api.github.c

In [6]:
GITHUB_TOKEN =  # Copy your GitHub token here
headers = {"Authorization": f"token {GITHUB_TOKEN}"}

In [38]:
import time
import math
from pathlib import Path
import pandas as pd
from tqdm.notebook import tqdm


def fetch_issues(
    owner="huggingface",
    repo="datasets",
    num_issues=10_000,
    rate_limit=5_000,
    issues_path=Path("."),
):
    if not issues_path.is_dir():
        issues_path.mkdir(exist_ok=True)

    batch = []
    all_issues = []
    per_page = 100  # Number of issues to return per page
    num_pages = math.ceil(num_issues / per_page)
    base_url = "https://api.github.com/repos"

    for page in tqdm(range(num_pages)):
        # Query with state=all to get both open and closed issues
        query = f"issues?page={page}&per_page={per_page}&state=all"
        issues = requests.get(f"{base_url}/{owner}/{repo}/{query}", headers=headers)
        batch.extend(issues.json())

        #if len(batch) > rate_limit and len(all_issues) < num_issues:
        #    all_issues.extend(batch)
        #    batch = []  # Flush batch for next time period
        #    print(f"Reached GitHub rate limit. Sleeping for one hour ...")
        #    break

    all_issues.extend(batch)
    df = pd.DataFrame.from_records(all_issues)
    df.to_json(f"{issues_path}/{repo}-issues.jsonl", orient="records", lines=True)
    print(
        f"Downloaded all the issues for {repo}! Dataset stored at {issues_path}/{repo}-issues.jsonl"
    )

In [39]:
fetch_issues()

  0%|          | 0/100 [00:00<?, ?it/s]

Downloaded all the issues for datasets! Dataset stored at ./datasets-issues.jsonl


In [40]:
from datasets import load_dataset
issues_dataset = load_dataset("json", data_files="datasets-issues.jsonl", split="train")
issues_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: ignored

In [41]:
import pandas as pd
from datasets import Dataset

df = pd.read_json('datasets-issues.jsonl', lines=True)
issues_dataset = Dataset.from_pandas(df)
#issues_dataset['train']

In [42]:
issues_dataset

Dataset({
    features: ['url', 'repository_url', 'labels_url', 'comments_url', 'events_url', 'html_url', 'id', 'node_id', 'number', 'title', 'user', 'labels', 'state', 'locked', 'assignee', 'assignees', 'milestone', 'comments', 'created_at', 'updated_at', 'closed_at', 'author_association', 'active_lock_reason', 'body', 'reactions', 'timeline_url', 'performed_via_github_app', 'state_reason', 'draft', 'pull_request'],
    num_rows: 6455
})

In [43]:
sample = issues_dataset.shuffle(seed=666).select(range(3))

for url, pr in zip(sample["html_url"], sample["pull_request"]):
    print(f">> URL: {url}")
    print(f">> Pull request: {pr}\n")

>> URL: https://github.com/huggingface/datasets/pull/28
>> Pull request: {'diff_url': 'https://github.com/huggingface/datasets/pull/28.diff', 'html_url': 'https://github.com/huggingface/datasets/pull/28', 'merged_at': '2020-04-30T19:51:08Z', 'patch_url': 'https://github.com/huggingface/datasets/pull/28.patch', 'url': 'https://api.github.com/repos/huggingface/datasets/pulls/28'}

>> URL: https://github.com/huggingface/datasets/issues/2768
>> Pull request: None

>> URL: https://github.com/huggingface/datasets/issues/6014
>> Pull request: None



In [35]:
issues_dataset = issues_dataset.map(
    lambda x: {"is_pull_request": False if x["pull_request"] is None else True}
)

Map:   0%|          | 0/5100 [00:00<?, ? examples/s]

In [37]:
issues_dataset[0]

{'url': 'https://api.github.com/repos/huggingface/datasets/issues/6483',
 'repository_url': 'https://api.github.com/repos/huggingface/datasets',
 'labels_url': 'https://api.github.com/repos/huggingface/datasets/issues/6483/labels{/name}',
 'comments_url': 'https://api.github.com/repos/huggingface/datasets/issues/6483/comments',
 'events_url': 'https://api.github.com/repos/huggingface/datasets/issues/6483/events',
 'html_url': 'https://github.com/huggingface/datasets/issues/6483',
 'id': 2032946981,
 'node_id': 'I_kwDODunzps55LE8l',
 'number': 6483,
 'title': 'Iterable Dataset: rename column clashes with remove column',
 'user': {'avatar_url': 'https://avatars.githubusercontent.com/u/93869735?v=4',
  'events_url': 'https://api.github.com/users/sanchit-gandhi/events{/privacy}',
  'followers_url': 'https://api.github.com/users/sanchit-gandhi/followers',
  'following_url': 'https://api.github.com/users/sanchit-gandhi/following{/other_user}',
  'gists_url': 'https://api.github.com/users/san

In [44]:
issue_number = 2792
url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
response = requests.get(url, headers=headers)
response.json()

[{'url': 'https://api.github.com/repos/huggingface/datasets/issues/comments/897594128',
  'html_url': 'https://github.com/huggingface/datasets/pull/2792#issuecomment-897594128',
  'issue_url': 'https://api.github.com/repos/huggingface/datasets/issues/2792',
  'id': 897594128,
  'node_id': 'IC_kwDODunzps41gDMQ',
  'user': {'login': 'bhavitvyamalik',
   'id': 19718818,
   'node_id': 'MDQ6VXNlcjE5NzE4ODE4',
   'avatar_url': 'https://avatars.githubusercontent.com/u/19718818?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/bhavitvyamalik',
   'html_url': 'https://github.com/bhavitvyamalik',
   'followers_url': 'https://api.github.com/users/bhavitvyamalik/followers',
   'following_url': 'https://api.github.com/users/bhavitvyamalik/following{/other_user}',
   'gists_url': 'https://api.github.com/users/bhavitvyamalik/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/bhavitvyamalik/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/

In [45]:
def get_comments(issue_number):
    url = f"https://api.github.com/repos/huggingface/datasets/issues/{issue_number}/comments"
    response = requests.get(url, headers=headers)
    return [r["body"] for r in response.json()]


# Test our function works as expected
get_comments(2792)

["@albertvillanova my tests are failing here:\r\n```\r\ndataset_name = 'gooaq'\r\n\r\n    def test_load_dataset(self, dataset_name):\r\n        configs = self.dataset_tester.load_all_configs(dataset_name, is_local=True)[:1]\r\n>       self.dataset_tester.check_load_dataset(dataset_name, configs, is_local=True, use_local_dummy_data=True)\r\n\r\ntests/test_dataset_common.py:234: \r\n_ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ \r\ntests/test_dataset_common.py:187: in check_load_dataset\r\n    self.parent.assertTrue(len(dataset[split]) > 0)\r\nE   AssertionError: False is not true\r\n```\r\nWhen I try loading dataset on local machine it works fine. Any suggestions on how can I avoid this error?",
 'Thanks for the help, @albertvillanova! All tests are passing now.']

In [ ]:
# Depending on your internet connection, this can take a few minutes...
issues_with_comments_dataset = issues_dataset.map(
    lambda x: {"comments": get_comments(x["number"])}
)

Map:   0%|          | 0/6455 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()


In [ ]:
issues_with_comments_dataset.push_to_hub('github-issues')